In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!git clone https://github.com/MStypulkowski/BiGAN.git
%cd BiGAN


Cloning into 'BiGAN'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39 (from 1)
Receiving objects: 100% (39/39), 4.20 MiB | 22.05 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/BiGAN


In [14]:
!python train_biggan.py


python3: can't open file '/content/BiGAN/train_biggan.py': [Errno 2] No such file or directory


In [15]:
!pip install torch torchvision


In [16]:
import os
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

class MedicalImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [file for file in os.listdir(root_dir) if file.lower().endswith(('jpg', 'jpeg', 'png'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image


In [17]:
import torch
import torch.nn as nn

# Generator
class Generator(nn.Module):
    def __init__(self, nz, ngf, nc):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # State: (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # State: (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # State: (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # State: (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # Output: (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

# Discriminator
class Discriminator(nn.Module):
    def __init__(self, nc, ndf):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # Input: (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # State: (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # State: (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # State: (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # State: (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
            # Output: Scalar probability
        )

    def forward(self, input):
        return self.main(input)


In [18]:
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader

# Hyperparameters
batch_size = 64
image_size = 64
nz = 100  # Size of z latent vector (i.e., size of generator input)
ngf = 64  # Size of feature maps in generator
ndf = 64  # Size of feature maps in discriminator
nc = 3    # Number of channels in the training images. For color images this is 3

# Data transformations
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Dataset and DataLoader
dataset = MedicalImageDataset(root_dir='/content/drive/MyDrive/dfu /PartB_DFU_Dataset/Infection/Aug-Negative', transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
netG = Generator(nz, ngf, nc).to(device)
netD = Discriminator(nc, ndf).to(device)

# Loss function
criterion = nn.BCELoss()

# Optimizers
optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))


In [21]:
import torch
import torchvision.utils as vutils
import os

# Create output directory
output_dir = '/content/drive/MyDrive/medical_gan_outputs'
os.makedirs(output_dir, exist_ok=True)

# Fixed noise for generating samples
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

num_epochs = 50

for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network
        ###########################
        ## Train with real batch
        netD.zero_grad()
        real_cpu = data.to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), 1., dtype=torch.float, device=device)
        output = netD(real_cpu).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with fake batch
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(0.)
        output = netD(fake.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network
        ###########################
        netG.zero_grad()
        label.fill_(1.)  # Generator wants discriminator to think outputs are real
        output = netD(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print(f'[{epoch}/{num_epochs}][{i}/{len(dataloader)}] '
                  f'Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f} '
                  f'D(x): {D_x:.4f} D(G(z)): {D_G_z1:.4f} / {D_G_z2:.4f}')

    # Save fake images for each epoch
    with torch.no_grad():
        fake = netG(fixed_noise).detach().cpu()
    vutils.save_image(fake, f'{output_dir}/fake_samples_epoch_{epoch+1}.png', normalize=True)

    # Save the model checkpoints
    torch.save(netG.state_dict(), f'{output_dir}/netG_epoch_{epoch+1}.pth')
    torch.save(netD.state_dict(), f'{output_dir}/netD_epoch_{epoch+1}.pth')


[0/50][0/47] Loss_D: 0.4258 Loss_G: 2.4712 D(x): 0.8137 D(G(z)): 0.1705 / 0.1130
[1/50][0/47] Loss_D: 0.7068 Loss_G: 3.4799 D(x): 0.8799 D(G(z)): 0.4180 / 0.0386
[2/50][0/47] Loss_D: 0.9790 Loss_G: 2.3194 D(x): 0.4427 D(G(z)): 0.0053 / 0.1286
[3/50][0/47] Loss_D: 2.2978 Loss_G: 3.1767 D(x): 0.9736 D(G(z)): 0.8450 / 0.0674
[4/50][0/47] Loss_D: 1.6245 Loss_G: 3.7270 D(x): 0.9423 D(G(z)): 0.7543 / 0.0333
[5/50][0/47] Loss_D: 1.0861 Loss_G: 2.1621 D(x): 0.5673 D(G(z)): 0.2571 / 0.1466
[6/50][0/47] Loss_D: 1.2001 Loss_G: 1.9313 D(x): 0.4046 D(G(z)): 0.0076 / 0.2065
[7/50][0/47] Loss_D: 1.8589 Loss_G: 1.7136 D(x): 0.3079 D(G(z)): 0.1239 / 0.3044
[8/50][0/47] Loss_D: 0.8771 Loss_G: 3.8911 D(x): 0.8879 D(G(z)): 0.4715 / 0.0333


KeyboardInterrupt: 